# Assignment 2 — DQN (in-place implementation)

This notebook implements a complete, well-documented DQN agent for OpenAI/Gymnasium environments (default: CartPole-v1).
It includes: environment setup, a replay buffer, Q-network, target network updates, training loop, evaluation, and Weights & Biases (WandB) logging.

Do not run any cells here unless you want to (you said you'll run tests). The cells are arranged so you can run them sequentially.

In [1]:
# Install dependencies (uncomment to run)
# !pip install gymnasium torch wandb numpy

## Imports and configuration
Small, self-contained utilities and device detection.

In [2]:
import os
import time
import random
from collections import deque, namedtuple
from typing import Tuple, List, Deque

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
import wandb

# Device detection
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## WandB login and experiment configuration
This cell reads an API key from `key.txt` (present in the folder) and initializes a WandB run.
If you prefer not to use WandB, set `USE_WANDB = False` below.

## 6. Hyperparameter Configuration

This cell contains all the hyperparameters for the DQN agent.
You can tweak the values here to see how they affect training performance for each environment. The assignment suggests experimenting with:
- `gamma` (Discount Factor)
- `epsilon_decay` (Epsilon Decay Rate)
- `lr` (NN Learning Rate)
- `buffer_size` (Replay Memory Size)
- `batch_size` (Learning Batch Size)

In [3]:
# --- Base Hyperparameter Configuration ---
# You can copy and modify this dictionary for each environment.
config = {
    # --- Agent ---
    "use_ddqn": True,             # Set to True to use Double DQN, False for standard DQN

    # --- Environment ---
    "env_name": "CartPole-v1",
    "seed": 42,

    # --- Training ---
    "max_episodes": 500,          # Number of episodes to train for
    "max_steps": 1000,            # Max steps per episode
    "start_training_after": 1000, # Steps to collect before training starts

    # --- DQN Agent ---
    "gamma": 0.99,                # Discount factor
    "lr": 1e-4,                   # Learning rate for the Adam optimizer
    "buffer_size": 10000,         # Max size of the replay buffer
    "batch_size": 64,             # Number of samples to train on from the buffer
    "target_update": 250,         # Steps between updating the target network

    # --- Epsilon-Greedy Exploration ---
    "epsilon_start": 1.0,         # Starting value of epsilon
    "epsilon_final": 0.05,        # Final value of epsilon
    "epsilon_decay": 10000,       # Epsilon decay rate
}

## Utilities: seeding and helpers
Deterministic seeds (best-effort) and a small helper for epsilon schedule.

In [4]:
def set_seed(seed: int, env=None):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    if env is not None:
        try:
            env.reset(seed=seed)
            env.action_space.seed(seed)
            env.observation_space.seed(seed)
        except Exception:
            pass

def linear_epsilon(start, final, decay, step):
    return final + (start - final) * np.exp(-1. * step / decay)

## Replay Buffer
A simple deque-based experience replay with sampling.

In [5]:
Transition = namedtuple('Transition', ['state', 'action', 'reward', 'next_state', 'done'])

class ReplayBuffer:
    def __init__(self, capacity: int):
        self.buffer: Deque[Transition] = deque(maxlen=capacity)

    def push(self, *args):
        self.buffer.append(Transition(*args))

    def sample(self, batch_size: int):
        batch = random.sample(self.buffer, batch_size)
        states = torch.tensor([b.state for b in batch], dtype=torch.float32, device=device)
        actions = torch.tensor([b.action for b in batch], dtype=torch.int64, device=device).unsqueeze(1)
        rewards = torch.tensor([b.reward for b in batch], dtype=torch.float32, device=device).unsqueeze(1)
        next_states = torch.tensor([b.next_state for b in batch], dtype=torch.float32, device=device)
        dones = torch.tensor([b.done for b in batch], dtype=torch.float32, device=device).unsqueeze(1)
        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.buffer)

## Q-network
A small MLP suitable for low-dimensional observations (e.g., CartPole).

In [6]:
class QNetwork(nn.Module):
    def __init__(self, obs_size: int, n_actions: int, hidden_sizes=(128, 128)):
        super().__init__()
        layers = []
        in_size = obs_size
        for h in hidden_sizes:
            layers.append(nn.Linear(in_size, h))
            layers.append(nn.ReLU())
            in_size = h
        layers.append(nn.Linear(in_size, n_actions))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

## DQN Agent
Includes epsilon-greedy policy, optimization step, and target network updates.

## 7. Training and Evaluation Loop

This section contains the core `train` and `evaluate` functions.

- **`train()`**: This function takes an agent and an environment, runs the main training loop, and logs the results. It now saves the trained model to a specific path for each environment.
- **`evaluate()`**: This function takes a trained agent and an environment and runs a set number of episodes with a greedy policy (no exploration) to measure its performance.

In [7]:
def train(agent, env, cfg, run_name):
    """
    Train a DQN agent on a given environment.

    Args:
        agent (DQNAgent): The agent to train.
        env (gym.Env): The environment to train on.
        cfg (dict): The hyperparameter configuration.
        run_name (str): The name for the WandB run.
    """
    if USE_WANDB:
        # Ensure any previous run is finished
        if wandb.run is not None:
            wandb.finish()
        # Start a new run
        wandb.init(
            project=PROJECT_NAME,
            name=run_name,
            config=cfg,
            reinit=True
        )

    total_steps = 0
    print(f"--- Starting Training for {cfg['env_name']} ---")

    for episode in range(1, cfg['max_episodes'] + 1):
        state, _ = env.reset()
        ep_reward = 0.0
        
        for t in range(cfg['max_steps']):
            action = agent.select_action(state, training=True)
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            agent.remember(state, action, reward, next_state, float(done))
            loss = None
            if agent.steps_done > cfg['start_training_after']:
                loss = agent.optimize()
            
            state = next_state
            ep_reward += reward
            total_steps += 1

            if total_steps % cfg['target_update'] == 0:
                agent.update_target()

            if done:
                break
        
        # Logging
        log_stats = {'episode': episode, 'reward': ep_reward, 'total_steps': total_steps}
        if loss is not None:
            log_stats['loss'] = loss
        
        if USE_WANDB:
            wandb.log(log_stats)
        
        if episode % 50 == 0:
            print(f"  Episode {episode}/{cfg['max_episodes']} | Reward: {ep_reward:.2f}")

    print(f"--- Training Finished for {cfg['env_name']} ---")
    
    # Save the trained policy
    model_path = f"dqn_{cfg['env_name']}.pth"
    torch.save(agent.policy_net.state_dict(), model_path)
    print(f"Model saved to {model_path}\n")

    if USE_WANDB:
        wandb.finish()

def evaluate(agent, env, n_episodes=100):
    """
    Evaluate a trained agent on an environment.
    """
    print(f"--- Evaluating for {env.spec.id} over {n_episodes} episodes ---")
    agent.policy_net.eval()  # Set network to evaluation mode
    
    total_rewards = []
    for episode in range(n_episodes):
        state, _ = env.reset()
        done = False
        truncated = False
        ep_reward = 0
        
        while not done and not truncated:
            action = agent.select_action(state, training=False)  # Greedy action
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            ep_reward += reward
            state = next_state
            
        total_rewards.append(ep_reward)

    avg_reward = np.mean(total_rewards)
    print(f"  Average Reward: {avg_reward:.2f}\n")
    return avg_reward

## 8. Main Execution Block

This is the final, runnable part of the notebook. It will:
1.  Define the list of environments to be trained and tested on.
2.  Loop through each environment.
3.  For each one, it creates the environment and a new DQN agent.
4.  Calls `train()` to train the agent on that specific environment.
5.  Calls `evaluate()` to test the newly trained agent for 100 episodes.
6.  Stores and prints the final evaluation results.

**This is the only cell you need to run to perform all training and testing.**

In [8]:
# --- List of Environments to Run ---
environments = [
    "CartPole-v1",
    "Acrobot-v1",
    "MountainCar-v0",
]

# --- Store final evaluation results ---
evaluation_results = {}

# --- Main Loop ---
for use_ddqn_flag in [False, True]: # Run for both DQN and DDQN
    algo_name = "DDQN" if use_ddqn_flag else "DQN"
    print(f"\n{'='*40}\nRunning Experiment with: {algo_name}\n{'='*40}\n")

    for env_name in environments:
        # 1. Create the environment
        env = gymnasium.make(env_name)
        
        # 2. Update the config for the current environment and algorithm
        current_config = config.copy()
        current_config["env_name"] = env_name
        current_config["use_ddqn"] = use_ddqn_flag
        
        # Adjust hyperparameters for specific environments if needed
        if env_name == "Acrobot-v1":
            current_config["max_episodes"] = 1000
            current_config["lr"] = 5e-4
            current_config["target_update"] = 500
        elif env_name == "MountainCar-v0":
            current_config["max_episodes"] = 2000
            current_config["epsilon_decay"] = 30000
            current_config["start_training_after"] = 2000
            current_config["lr"] = 1e-3

        # 3. Create the agent
        obs_size = env.observation_space.shape[0]
        action_size = env.action_space.n
        agent = DQNAgent(obs_size, action_size, current_config)

        # 4. Set a unique run name for WandB
        run_name = f"{algo_name}_{env_name}_{time.strftime('%Y%m%d-%H%M%S')}"

        # 5. Train the agent
        set_seed(current_config['seed'], env)
        train(agent, env, current_config, run_name)

        # 6. Evaluate the trained agent
        avg_reward = evaluate(agent, env, n_episodes=100)
        
        # Store results in a structured way
        if algo_name not in evaluation_results:
            evaluation_results[algo_name] = {}
        evaluation_results[algo_name][env_name] = avg_reward
        
        # 7. Close the environment
        env.close()

# --- Final Summary ---
print("\n\n--- Overall Evaluation Summary ---")
for algo_name, results in evaluation_results.items():
    print(f"\n--- {algo_name} Results ---")
    for env_name, avg_reward in results.items():
        print(f"  Environment: {env_name} | Average Reward (100 eps): {avg_reward:.2f}")
print("------------------------------------")


Running Experiment with: DQN



NameError: name 'gymnasium' is not defined

In [ ]:
class DQNAgent:
    def __init__(self, obs_size, n_actions, cfg):
        self.obs_size = obs_size
        self.n_actions = n_actions
        self.cfg = cfg
        self.use_ddqn = cfg.get("use_ddqn", False) # Default to standard DQN

        self.policy_net = QNetwork(obs_size, n_actions).to(device)
        self.target_net = QNetwork(obs_size, n_actions).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=cfg['lr'])
        self.replay = ReplayBuffer(cfg['buffer_size'])
        self.steps_done = 0

    def select_action(self, state, training=True):
        # state: np.array or torch tensor (1d)
        epsilon = linear_epsilon(self.cfg['epsilon_start'], self.cfg['epsilon_final'], self.cfg['epsilon_decay'], self.steps_done)
        
        # Only increment steps during training
        if training:
            self.steps_done += 1

        if training and random.random() < epsilon:
            return random.randrange(self.n_actions)
        else:
            with torch.no_grad():
                s = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
                qvals = self.policy_net(s)
                return int(qvals.argmax(dim=1).item())

    def remember(self, state, action, reward, next_state, done):
        self.replay.push(state, action, reward, next_state, done)

    def update_target(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())

    def optimize(self):
        if len(self.replay) < self.cfg['batch_size']:
            return None
        states, actions, rewards, next_states, dones = self.replay.sample(self.cfg['batch_size'])

        # Q(s,a) - The Q-value for the action that was actually taken
        q_values = self.policy_net(states).gather(1, actions)

        with torch.no_grad():
            if self.use_ddqn:
                # --- Double DQN ---
                # 1. Select the best action a' from the *policy* network for the next state s'
                next_actions = self.policy_net(next_states).argmax(dim=1).unsqueeze(1)
                # 2. Evaluate that action a' using the *target* network to get Q(s', a')
                next_q_values = self.target_net(next_states).gather(1, next_actions)
            else:
                # --- Standard DQN ---
                # Select the max Q-value from the target network for the next state
                next_q_values = self.target_net(next_states).max(1)[0].unsqueeze(1)
            
            # Calculate the target Q-value: r + gamma * Q_target(s', a') * (1 - done)
            target = rewards + (1 - dones) * (self.cfg['gamma'] * next_q_values)

        loss = nn.functional.mse_loss(q_values, target)
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0) # Clip gradients
        self.optimizer.step()
        
        return loss.item()

## Training loop (no execution)
This cell contains the train() function which runs episodes, logs metrics, and updates networks.

## Evaluation helper
Run deterministic episodes using the policy network (no exploration).

## How to run (suggested)
Run the following steps in order in your environment (PowerShell on Windows):

In [ ]:
# Example (do not run automatically in this notebook):
# 1) Create and activate a virtualenv (recommended)
# powershell: python -m venv .venv; .\.venv\Scripts\Activate.ps1
# 2) Install dependencies
# pip install gymnasium torch wandb numpy
# 3) (Optional) edit `config` dictionary above for hyperparameters
# 4) Run training:
# result = train()
# 5) Evaluate:
# rewards = evaluate(torch.load('dqn_policy.pth'))
# print(rewards)

print('Notebook ready. Run the cells step-by-step in your environment.')

Notebook ready. Run the cells step-by-step in your environment.


## Notes and tips
- The default configuration uses CartPole-v1; change `config['env_name']` to try other Gymnasium envs.
- WandB will log metrics if `USE_WANDB=True` and `key.txt` contains a valid key.
- I kept the implementation compact and readable; feel free to split cells or extract modules into `.py` files.
- You asked me not to run tests; I only modified the notebook in-place. Run training locally when ready.